In [1]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 2.5 MB/s 


In [2]:
import pandas as pd
import numpy as np
import random
import category_encoders as ce
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/NYPD_Complaint_Data_Historic.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6983207 entries, 0 to 6983206
Data columns (total 35 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CMPLNT_NUM         int64  
 1   CMPLNT_FR_DT       object 
 2   CMPLNT_FR_TM       object 
 3   CMPLNT_TO_DT       object 
 4   CMPLNT_TO_TM       object 
 5   ADDR_PCT_CD        float64
 6   RPT_DT             object 
 7   KY_CD              int64  
 8   OFNS_DESC          object 
 9   PD_CD              float64
 10  PD_DESC            object 
 11  CRM_ATPT_CPTD_CD   object 
 12  LAW_CAT_CD         object 
 13  BORO_NM            object 
 14  LOC_OF_OCCUR_DESC  object 
 15  PREM_TYP_DESC      object 
 16  JURIS_DESC         object 
 17  JURISDICTION_CODE  float64
 18  PARKS_NM           object 
 19  HADEVELOPT         object 
 20  HOUSING_PSA        object 
 21  X_COORD_CD         float64
 22  Y_COORD_CD         float64
 23  SUSP_AGE_GROUP     object 
 24  SUSP_RACE          object 
 25  SUSP_SEX          

In [5]:
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,700381962,05/28/2015,15:00:00,NaN,NaN,46.0,06/01/2015,578,HARRASSMENT 2,638.0,...,M,NaN,40.845868,-73.915888,"(40.84586773, -73.915888033)",PATROL BORO BRONX,NaN,25-44,WHITE HISPANIC,F
1,642234217,10/28/2013,13:50:00,10/28/2013,13:50:00,120.0,10/28/2013,351,CRIMINAL MISCHIEF & RELATED OF,259.0,...,NaN,NaN,40.627061,-74.077149,"(40.627060894, -74.077149232)",PATROL BORO STATEN ISLAND,NaN,45-64,WHITE,M
2,242465164,05/09/2012,20:50:00,05/09/2012,21:00:00,24.0,05/09/2012,236,DANGEROUS WEAPONS,782.0,...,NaN,NaN,40.800966,-73.969047,"(40.800965968, -73.969047272)",PATROL BORO MAN NORTH,NaN,NaN,UNKNOWN,E
3,927207428,01/03/2014,13:30:00,01/03/2014,13:35:00,108.0,01/03/2014,109,GRAND LARCENY,409.0,...,M,NaN,40.745242,-73.894253,"(40.745241809, -73.894253382)",PATROL BORO QUEENS NORTH,NaN,45-64,ASIAN / PACIFIC ISLANDER,M
4,492142357,04/13/2016,00:00:00,NaN,NaN,40.0,04/13/2016,351,CRIMINAL MISCHIEF & RELATED OF,258.0,...,NaN,NaN,40.810352,-73.924942,"(40.810351863, -73.924942326)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E


In [6]:
df = df[['CMPLNT_FR_DT','CMPLNT_FR_TM','VIC_SEX','VIC_RACE','VIC_AGE_GROUP','Latitude','Longitude','BORO_NM','OFNS_DESC','KY_CD']]

In [7]:
df = df.dropna()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5309755 entries, 0 to 6983204
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   CMPLNT_FR_DT   object 
 1   CMPLNT_FR_TM   object 
 2   VIC_SEX        object 
 3   VIC_RACE       object 
 4   VIC_AGE_GROUP  object 
 5   Latitude       float64
 6   Longitude      float64
 7   BORO_NM        object 
 8   OFNS_DESC      object 
 9   KY_CD          int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 445.6+ MB


In [9]:
df['time'] = pd.to_datetime(df['CMPLNT_FR_TM'], format="%H:%M:%S")

In [10]:
df["hour"] = df['time'].dt.hour

In [11]:
df['Date'] = pd.to_datetime(df['CMPLNT_FR_DT'], errors = 'coerce')

In [13]:
def downcast(df): 
    column = df.dtypes.index.tolist() 
    types = df.dtypes.values.tolist() 
    for i in range(len(types)):
        if 'int' in str(types[i]): 
            if df[column[i]].min() > -128 and df[column[i]].max() < 128: df[column[i]] = df[column[i]].astype('int8') 
            elif df[column[i]].min() > -32768 and df[column[i]].max() < 32767: df[column[i]] = df[column[i]].astype('int16')
            elif df[column[i]].min() > -2147483648 and df[column[i]].max() < 2147483647: df[column[i]] = df[column[i]].astype('int32')
            else: df[column[i]] = df[column[i]].astype(np.int64) 
        elif 'float' in str(types[i]):
            if df[column[i]].min() > np.finfo('float16').min and df[column[i]].max() < np.finfo('float16').max: df[column[i]] = df[column[i]].astype('float16') 
            elif df[column[i]].min() > np.finfo('float32').min and df[column[i]].max() < np.finfo('float32').max: df[column[i]] = df[column[i]].astype('float32')
            else: df[column[i]] = df[column[i]].astype('float64') 
        elif types[i] == np.object:
                if column[i] == 'date': df[column[i]] = pd.to_datetime(df[column[i]], format='%Y-%m-%d')
                else: df[column[i]] = df[column[i]].astype('category') 
    return df  

In [14]:
df = downcast(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [15]:
df.info(verbose = True, null_counts = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5309755 entries, 0 to 6983204
Data columns (total 13 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   CMPLNT_FR_DT   5309755 non-null  category      
 1   CMPLNT_FR_TM   5309755 non-null  category      
 2   VIC_SEX        5309755 non-null  category      
 3   VIC_RACE       5309755 non-null  category      
 4   VIC_AGE_GROUP  5309755 non-null  category      
 5   Latitude       5309755 non-null  float16       
 6   Longitude      5309755 non-null  float16       
 7   BORO_NM        5309755 non-null  category      
 8   OFNS_DESC      5309755 non-null  category      
 9   KY_CD          5309755 non-null  int16         
 10  time           5309755 non-null  datetime64[ns]
 11  hour           5309755 non-null  int8          
 12  Date           5309614 non-null  datetime64[ns]
dtypes: category(7), datetime64[ns](2), float16(2), int16(1), int8(1)
memory usage: 208.0 MB

In [16]:
df_copy = df.copy()

In [17]:
df = df[df['Date'].notna()]

In [18]:
df.head()

,CMPLNT_FR_DT,CMPLNT_FR_TM,VIC_SEX,VIC_RACE,VIC_AGE_GROUP,Latitude,Longitude,BORO_NM,OFNS_DESC,KY_CD,time,hour,Date
0,05/28/2015,15:00:00,F,WHITE HISPANIC,25-44,40.84375,-73.9375,BRONX,HARRASSMENT 2,578,1900-01-01 15:00:00,15,2015-05-28
1,10/28/2013,13:50:00,M,WHITE,45-64,40.62500,-74.0625,STATEN ISLAND,CRIMINAL MISCHIEF & RELATED OF,351,1900-01-01 13:50:00,13,2013-10-28
3,01/03/2014,13:30:00,M,ASIAN / PACIFIC ISLANDER,45-64,40.75000,-73.8750,QUEENS,GRAND LARCENY,109,1900-01-01 13:30:00,13,2014-01-03
4,04/13/2016,00:00:00,E,UNKNOWN,UNKNOWN,40.81250,-73.9375,BRONX,CRIMINAL MISCHIEF & RELATED OF,351,1900-01-01 00:00:00,0,2016-04-13
5,08/18/2014,14:30:00,F,WHITE,18-24,40.68750,-73.8750,QUEENS,PETIT LARCENY,341,1900-01-01 14:30:00,14,2014-08-18


In [19]:
df["month"] = df['Date'].dt.month
df["day_of_week"] = df['Date'].dt.weekday

In [20]:
df.VIC_AGE_GROUP.unique()

['25-44', '45-64', 'UNKNOWN', '18-24', '65+', ..., '1017', '-51', '972', '-59', '-5']
Length: 196
Categories (196, object): ['-1', '-2', '-3', '-32', ..., '983', '999', '<18', 'UNKNOWN']

In [21]:
df.VIC_AGE_GROUP = df.VIC_AGE_GROUP.replace('UNKNOWN', np.nan)

In [22]:
df = df.loc[df.VIC_AGE_GROUP.isin(['25-44','18-24','45-64','65+','<18'])]

In [23]:
df.VIC_AGE_GROUP.unique()

['25-44', '45-64', '18-24', '65+', '<18']
Categories (195, object): ['-1', '-2', '-3', '-32', ..., '981', '983', '999', '<18']

In [24]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4720543 entries, 0 to 6983204
Data columns (total 15 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   CMPLNT_FR_DT   4720543 non-null  category      
 1   CMPLNT_FR_TM   4720543 non-null  category      
 2   VIC_SEX        4720543 non-null  category      
 3   VIC_RACE       4720543 non-null  category      
 4   VIC_AGE_GROUP  4720543 non-null  category      
 5   Latitude       4720543 non-null  float16       
 6   Longitude      4720543 non-null  float16       
 7   BORO_NM        4720543 non-null  category      
 8   OFNS_DESC      4720543 non-null  category      
 9   KY_CD          4720543 non-null  int16         
 10  time           4720543 non-null  datetime64[ns]
 11  hour           4720543 non-null  int8          
 12  Date           4720543 non-null  datetime64[ns]
 13  month          4720543 non-null  int64         
 14  day_of_week    4720543 non-null  i

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [25]:
(df.OFNS_DESC.unique().tolist())


['HARRASSMENT 2',
 'CRIMINAL MISCHIEF & RELATED OF',
 'GRAND LARCENY',
 'PETIT LARCENY',
 'ASSAULT 3 & RELATED OFFENSES',
 'FORGERY',
 'OFF. AGNST PUB ORD SENSBLTY &',
 'FELONY ASSAULT',
 'VEHICLE AND TRAFFIC LAWS',
 'OFFENSES AGAINST PUBLIC ADMINI',
 'ROBBERY',
 'RAPE',
 'BURGLARY',
 'GRAND LARCENY OF MOTOR VEHICLE',
 'POSSESSION OF STOLEN PROPERTY',
 'THEFT-FRAUD',
 'MISCELLANEOUS PENAL LAW',
 'CRIMINAL TRESPASS',
 'DANGEROUS WEAPONS',
 'ADMINISTRATIVE CODE',
 'UNAUTHORIZED USE OF A VEHICLE',
 'OTHER OFFENSES RELATED TO THEF',
 'FRAUDS',
 'SEX CRIMES',
 'ARSON',
 'OFFENSES AGAINST THE PERSON',
 "BURGLAR'S TOOLS",
 'FRAUDULENT ACCOSTING',
 'INTOXICATED & IMPAIRED DRIVING',
 'DANGEROUS DRUGS',
 'OTHER STATE LAWS',
 'JOSTLING',
 'OFFENSES INVOLVING FRAUD',
 'PETIT LARCENY OF MOTOR VEHICLE',
 'ENDAN WELFARE INCOMP',
 'KIDNAPPING & RELATED OFFENSES',
 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
 'PROSTITUTION & RELATED OFFENSES',
 'OTHER STATE LAWS (NON PENAL LA',
 'CHILD ABANDONMENT/NON SUPP

In [26]:
len(df.KY_CD.unique()) 

70

In [27]:
KY_OFNS = df_copy.copy()

In [28]:
KY_OFNS = KY_OFNS[['KY_CD','OFNS_DESC']]

In [29]:
KY_OFNS = KY_OFNS.drop_duplicates()

In [30]:
len(KY_OFNS)

82

In [31]:
KY_OFNS = KY_OFNS.replace(to_replace =["DANGEROUS DRUGS", "INTOXICATED/IMPAIRED DRIVING","UNDER THE INFLUENCE OF DRUGS","LOITERING FOR DRUG PURPOSES","INTOXICATED & IMPAIRED DRIVING","ALCOHOLIC BEVERAGE CONTROL LAW"],  
                            value ="Drug and Alchohol crimes") 

In [32]:
KY_OFNS = KY_OFNS.replace(to_replace =[360,577,117,347,235,346,119], value = 1)

In [33]:
KY_OFNS = KY_OFNS.replace(to_replace =["GRAND LARCENY","PETIT LARCENY OF MOTOR VEHICLE","PETIT LARCENY","ROBBERY","BURGLARY","GRAND LARCENY OF MOTOR VEHICLE","THEFT-FRAUD","POSSESSION OF STOLEN PROPERTY","OTHER OFFENSES RELATED TO THEF","THEFT OF SERVICES","THEFT OF SERVICES"], value = "Theft")

In [34]:
KY_OFNS = KY_OFNS.replace(to_replace =[109,341,105,107,110,112,111,343,342], value = 2)

In [35]:
KY_OFNS = KY_OFNS.drop_duplicates()

In [36]:
len(KY_OFNS)

67

In [37]:
KY_OFNS

,KY_CD,OFNS_DESC
0,578,HARRASSMENT 2
1,351,CRIMINAL MISCHIEF & RELATED OF
3,2,Theft
6,344,ASSAULT 3 & RELATED OFFENSES
9,113,FORGERY
...,...,...
781073,362,OFFENSES AGAINST MARRIAGE UNCL
962910,366,NEW YORK CITY HEALTH CODE
1070778,123,ABORTION
1089405,357,FORTUNE TELLING


In [38]:
KY_OFNS = KY_OFNS.replace(to_replace =["RAPE", "PROSTITUTION & RELATED OFFENSES","FELONY SEX CRIMES"],  
                            value ="SEX CRIMES")

In [39]:
KY_OFNS = KY_OFNS.replace(to_replace =[104,233,116,356,115,116,119,234], value =3)

In [40]:
KY_OFNS = KY_OFNS.replace(to_replace =["ASSAULT 3 & RELATED OFFENSES", "FELONY ASSAULT","HOMICIDE-NEGLIGENT,UNCLASSIFIE","HOMICIDE-NEGLIGENT-VEHICLE","MURDER"],  
                            value ="Assault or Murder")

In [41]:
KY_OFNS = KY_OFNS.replace(to_replace =[344,106,103,102,101], value =4)

In [42]:
KY_OFNS = KY_OFNS.drop_duplicates()

In [43]:
len(KY_OFNS)

58

In [44]:
KY_OFNS = KY_OFNS.replace(to_replace =["ENDAN WELFARE INCOMP","ABORTION", "KIDNAPPING AND RELATED OFFENSES","KIDNAPPING","OFFENSES RELATED TO CHILDREN","CHILD ABANDONMENT/NON SUPPORT","KIDNAPPING & RELATED OFFENSES"],  
                            value ="Kidnapping and other crimes against children")

In [45]:
KY_OFNS = KY_OFNS.replace(to_replace =[123,124,345], value =5)

In [46]:
KY_OFNS = KY_OFNS.replace(to_replace =["OTHER STATE LAWS","AGRICULTURE & MRKTS LAW-UNCLASSIFIED","ADMINISTRATIVE CODES", "NEW YORK CITY HEALTH CODE","OTHER TRAFFIC INFRACTION","OTHER STATE LAWS (NON PENAL LAW)","NYS LAWS-UNCLASSIFIED VIOLATION","ADMINISTRATIVE CODE","OTHER STATE LAWS (NON PENAL LA","NYS LAWS-UNCLASSIFIED FELONY","OFF. AGNST PUB ORD SENSBLTY &","VEHICLE AND TRAFFIC LAWS","OFFENSES AGAINST PUBLIC ADMINI","ADMINISTRATIVE CODE"],  
                            value ="City Law Violation")

In [47]:
KY_OFNS = KY_OFNS.replace(to_replace =[685,366,881,364,676,677,675,125,364,361,348,359,365], value =6)

In [48]:
KY_OFNS = KY_OFNS.drop_duplicates()

In [49]:
(KY_OFNS)

,KY_CD,OFNS_DESC
0,578,HARRASSMENT 2
1,351,CRIMINAL MISCHIEF & RELATED OF
3,2,Theft
6,4,Assault or Murder
9,113,FORGERY
12,1,Drug and Alchohol crimes
16,121,CRIMINAL MISCHIEF & RELATED OF
17,236,DANGEROUS WEAPONS
20,6,City Law Violation
40,118,DANGEROUS WEAPONS


In [50]:
KY_OFNS = KY_OFNS.loc[KY_OFNS.KY_CD.isin([1,2,3,4,5,6])]

In [51]:
KY_OFNS = KY_OFNS.drop_duplicates()
KY_OFNS

,KY_CD,OFNS_DESC
3,2,Theft
6,4,Assault or Murder
12,1,Drug and Alchohol crimes
20,6,City Law Violation
59,3,SEX CRIMES
6623,5,Kidnapping and other crimes against children


In [52]:
df = df.replace(to_replace =[360,577,117,347,235,346,119], value = 1)

In [53]:
df = df.replace(to_replace =[109,341,105,107,110,112,111,343,342], value = 2)

In [54]:
df = df.replace(to_replace =[104,233,116,356,115,116,119,234], value =3)

In [55]:
df = df.replace(to_replace =[344,106,103,102,101], value =4)

In [56]:
df= df.replace(to_replace =[123,124,345], value =5)

In [57]:
df = df.replace(to_replace =[685,366,881,364,676,677,675,125,364,361,348,359,365], value =6)

In [59]:
df.head()

,CMPLNT_FR_DT,CMPLNT_FR_TM,VIC_SEX,VIC_RACE,VIC_AGE_GROUP,Latitude,Longitude,BORO_NM,OFNS_DESC,KY_CD,time,hour,Date,month,day_of_week
0,05/28/2015,15:00:00,F,WHITE HISPANIC,25-44,40.84375,-73.9375,BRONX,HARRASSMENT 2,578,1900-01-01 15:00:00,15,2015-05-28,5,3
1,10/28/2013,13:50:00,M,WHITE,45-64,40.62500,-74.0625,STATEN ISLAND,CRIMINAL MISCHIEF & RELATED OF,351,1900-01-01 13:50:00,13,2013-10-28,10,0
3,01/03/2014,13:30:00,M,ASIAN / PACIFIC ISLANDER,45-64,40.75000,-73.8750,QUEENS,GRAND LARCENY,2,1900-01-01 13:30:00,13,2014-01-03,1,4
5,08/18/2014,14:30:00,F,WHITE,18-24,40.68750,-73.8750,QUEENS,PETIT LARCENY,2,1900-01-01 14:30:00,14,2014-08-18,8,0
6,02/20/2012,01:30:00,F,BLACK,25-44,40.81250,-73.9375,MANHATTAN,ASSAULT 3 & RELATED OFFENSES,4,1900-01-01 01:30:00,1,2012-02-20,2,0


In [60]:
df = df.loc[df.KY_CD.isin([1,2,3,4,5,6])]

In [61]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3259240 entries, 3 to 6983202
Data columns (total 15 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   CMPLNT_FR_DT   3259240 non-null  category      
 1   CMPLNT_FR_TM   3259240 non-null  category      
 2   VIC_SEX        3259240 non-null  category      
 3   VIC_RACE       3259240 non-null  category      
 4   VIC_AGE_GROUP  3259240 non-null  category      
 5   Latitude       3259240 non-null  float16       
 6   Longitude      3259240 non-null  float16       
 7   BORO_NM        3259240 non-null  category      
 8   OFNS_DESC      3259240 non-null  category      
 9   KY_CD          3259240 non-null  int64         
 10  time           3259240 non-null  datetime64[ns]
 11  hour           3259240 non-null  int8          
 12  Date           3259240 non-null  datetime64[ns]
 13  month          3259240 non-null  int64         
 14  day_of_week    3259240 non-null  i

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [62]:
df = df.drop(["CMPLNT_FR_DT","CMPLNT_FR_TM","OFNS_DESC","Date","time"], axis=1)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3259240 entries, 3 to 6983202
Data columns (total 10 columns):
 #   Column         Dtype   
---  ------         -----   
 0   VIC_SEX        category
 1   VIC_RACE       category
 2   VIC_AGE_GROUP  category
 3   Latitude       float16 
 4   Longitude      float16 
 5   BORO_NM        category
 6   KY_CD          int64   
 7   hour           int8    
 8   month          int64   
 9   day_of_week    int64   
dtypes: category(4), float16(2), int64(3), int8(1)
memory usage: 130.6 MB


In [64]:
df.to_csv(r'Users\akshitagulati\Downloads\BE Project\FinalDB.csv')

In [65]:
#converting to numeric for the model
df = df.replace(to_replace =["BRONX"], value = 0)
df = df.replace(to_replace =["BROOKLYN"], value = 1)
df = df.replace(to_replace =["STATEN ISLAND"], value = 2)
df = df.replace(to_replace =["MANHATTAN"], value = 3)
df = df.replace(to_replace =["QUEENS"], value = 4)

In [66]:
df.head()

,VIC_SEX,VIC_RACE,VIC_AGE_GROUP,Latitude,Longitude,BORO_NM,KY_CD,hour,month,day_of_week
3,M,ASIAN / PACIFIC ISLANDER,45-64,40.75000,-73.8750,4,2,13,1,4
5,F,WHITE,18-24,40.68750,-73.8750,4,2,14,8,0
6,F,BLACK,25-44,40.81250,-73.9375,3,4,1,2,0
8,M,WHITE,25-44,40.62500,-73.9375,1,2,16,9,6
15,F,BLACK HISPANIC,25-44,40.84375,-73.8750,0,4,13,12,0


In [67]:
(df.VIC_SEX.unique().tolist())

['M', 'F', 'D', 'E']

In [68]:
(df.VIC_RACE.unique().tolist())

['ASIAN / PACIFIC ISLANDER',
 'WHITE',
 'BLACK',
 'BLACK HISPANIC',
 'WHITE HISPANIC',
 'UNKNOWN',
 'AMERICAN INDIAN/ALASKAN NATIVE',
 'OTHER']

In [70]:
df = df.replace(to_replace =["ASIAN / PACIFIC ISLANDER"], value = 0)
df = df.replace(to_replace =["WHITE"], value = 1)
df = df.replace(to_replace =["BLACK"], value = 2)
df = df.replace(to_replace =["BLACK HISPANIC"], value = 3)
df = df.replace(to_replace =["WHITE HISPANIC"], value = 4)
df = df.replace(to_replace =["UNKNOWN"], value = 5)
df = df.replace(to_replace =["AMERICAN INDIAN/ALASKAN NATIVE"], value = 6)
df = df.replace(to_replace =["OTHER"], value = 7)

In [71]:
df = df.replace(to_replace =["M"], value = 0)
df = df.replace(to_replace =["F"], value = 1)
df = df.replace(to_replace =["D"], value = 2)
df = df.replace(to_replace =["E"], value = 3)

In [72]:
df = df.replace(to_replace =["25-44"], value = 2)
df = df.replace(to_replace =["18-24"], value = 1)
df = df.replace(to_replace =["45-64"], value = 3)
df = df.replace(to_replace =["65+"], value = 4)
df = df.replace(to_replace =["<18"], value = 0)

In [75]:
df['Latitude'] = df['Latitude'].apply(pd.to_numeric, errors='coerce')
df['Longitude'] = df['Longitude'].apply(pd.to_numeric, errors='coerce')


In [76]:
df.head()

,VIC_SEX,VIC_RACE,VIC_AGE_GROUP,Latitude,Longitude,BORO_NM,KY_CD,hour,month,day_of_week
3,0,0,3,40.75000,-73.8750,4,2,13,1,4
5,1,1,1,40.68750,-73.8750,4,2,14,8,0
6,1,2,2,40.81250,-73.9375,3,4,1,2,0
8,0,1,2,40.62500,-73.9375,1,2,16,9,6
15,1,3,2,40.84375,-73.8750,0,4,13,12,0


In [77]:
(df.month.unique().tolist())

[1, 8, 2, 9, 12, 5, 6, 10, 7, 4, 3, 11]

In [78]:
# Labels are the values we want to predict
labels = np.array(df['KY_CD'])

# Remove the crimecode from the dataframe to leave us with just the predictive features
features= df.drop('KY_CD', axis = 1)

# Save feature names for later use
feature_list = list(features.columns)

# Convert the features dataframe to a numpy array
features = np.array(features)

print(labels)
print(feature_list)

[2 2 4 ... 2 4 4]
['VIC_SEX', 'VIC_RACE', 'VIC_AGE_GROUP', 'Latitude', 'Longitude', 'BORO_NM', 'hour', 'month', 'day_of_week']


In [79]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20, random_state = 42)

In [80]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Training Features Shape: (2607392, 9)
Training Labels Shape: (2607392,)
Testing Features Shape: (651848, 9)
Testing Labels Shape: (651848,)


In [81]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

"# Get numerical feature importances\nimportances = list(rf.feature_importances_)\n# List of tuples with variable and importance\nfeature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]\n# Sort the feature importances by most important first\nfeature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)\n# Print out the feature and importances \n[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];"

In [82]:
rf_most_important = RandomForestClassifier(n_estimators=20)
# Extract the two most important features
important_indices = [feature_list.index('month'), feature_list.index('hour')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]
# Train the random forest
rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

"rf_most_important = RandomForestClassifier(n_estimators=20)\n# Extract the two most important features\nimportant_indices = [feature_list.index('month'), feature_list.index('hour')]\ntrain_important = train_features[:, important_indices]\ntest_important = test_features[:, important_indices]\n# Train the random forest\nrf_most_important.fit(train_important, train_labels)\n# Make predictions and determine the error\npredictions = rf_most_important.predict(test_important)\nerrors = abs(predictions - test_labels)\n# Display the performance metrics\nprint('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')\nmape = np.mean(100 * (errors / test_labels))\naccuracy = 100 - mape\nprint('Accuracy:', round(accuracy, 2), '%.')"

In [83]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
'bootstrap': [True],
'max_depth': [80, 90, 100, 110],
'max_features': [2, 3],
'min_samples_leaf': [3, 4, 5],
'min_samples_split': [8, 10, 12],
'n_estimators': [20, 100, 200]
}
# Create a based model
rf2 = RandomForestClassifier
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf2, param_grid = param_grid, 
cv = 3, n_jobs = -1, verbose = 2)

"from sklearn.model_selection import GridSearchCV\n# Create the parameter grid based on the results of random search \nparam_grid = {\n'bootstrap': [True],\n'max_depth': [80, 90, 100, 110],\n'max_features': [2, 3],\n'min_samples_leaf': [3, 4, 5],\n'min_samples_split': [8, 10, 12],\n'n_estimators': [20, 100, 200]\n}\n# Create a based model\nrf2 = RandomForestClassifier\n# Instantiate the grid search model\ngrid_search = GridSearchCV(estimator = rf2, param_grid = param_grid, \ncv = 3, n_jobs = -1, verbose = 2)"

In [84]:
# Fit the grid search to the data
grid_search.fit(train_features, train_labels)
grid_search.best_params_


'# Fit the grid search to the data\ngrid_search.fit(train_features, train_labels)\ngrid_search.best_params_\n'

In [85]:
best_grid = grid_search.best_estimator_
best_grid

'best_grid = grid_search.best_estimator_\nbest_grid'

In [86]:
rf3 = best_grid.fit(train_features, train_labels)

'rf3 = best_grid.fit(train_features, train_labels)'

In [87]:
predictions = rf3.predict(test_features)

errors = abs(predictions - test_labels)

# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape

print('Accuracy:', round(accuracy, 2), '%.')

"predictions = rf3.predict(test_features)\n\nerrors = abs(predictions - test_labels)\n\n# Display the performance metrics\nprint('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')\n\nmape = np.mean(100 * (errors / test_labels))\naccuracy = 100 - mape\n\nprint('Accuracy:', round(accuracy, 2), '%.')"

In [88]:
from sklearn.ensemble import RandomForestClassifier

In [89]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200 , num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

"from sklearn.model_selection import RandomizedSearchCV\n# Number of trees in random forest\nn_estimators = [int(x) for x in np.linspace(start = 20, stop = 200 , num = 10)]\n# Number of features to consider at every split\nmax_features = ['auto', 'sqrt']\n# Maximum number of levels in tree\nmax_depth = [int(x) for x in np.linspace(10, 110, num = 11)]\nmax_depth.append(None)\n# Minimum number of samples required to split a node\nmin_samples_split = [2, 5, 10]\n# Minimum number of samples required at each leaf node\nmin_samples_leaf = [1, 2, 4]\n# Method of selecting samples for training each tree\nbootstrap = [True, False]\n# Create the random grid\nrandom_grid = {'n_estimators': n_estimators,\n               'max_features': max_features,\n               'max_depth': max_depth,\n               'min_samples_split': min_samples_split,\n               'min_samples_leaf': min_samples_leaf,\n               'bootstrap': bootstrap}\nprint(random_grid)"

In [90]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)

'# Use the random grid to search for best hyperparameters\n# First create the base model to tune\nrf = RandomForestClassifier()\n# Random search of parameters, using 3 fold cross validation, \n# search across 100 different combinations, and use all available cores\nrf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)\n# Fit the random search model\nrf_random.fit(train_features, train_labels)'

In [91]:
rf_random.best_params_
rf_random.best_params_
{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 100}

"rf_random.best_params_\nrf_random.best_params_\n{'bootstrap': True,\n 'max_depth': 10,\n 'max_features': 'sqrt',\n 'min_samples_leaf': 2,\n 'min_samples_split': 10,\n 'n_estimators': 100}"

In [92]:
rf_best = RandomForestClassifier(bootstrap =  True, max_depth = 10, max_features = 'sqrt', min_samples_leaf = 2, min_samples_split = 10, n_estimators = 100)

In [93]:
rf_best.fit(train_features, train_labels)

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=10)

In [94]:
predictions = rf_best.predict(test_features)

errors = abs(predictions - test_labels)

# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape

print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 1.05 degrees.
Accuracy: 74.65 %.


In [95]:
#feature_list

In [103]:
Test_value = [1, 0, 4, 40.75, -73.875, 3, 1, 4, 1]
type(Test_value)

list

In [104]:
Test_value = np.array([1, 0, 4, 40.75, -73.875, 3, 1, 4, 1])


In [105]:
type(Test_value)
New_value = Test_value.reshape(1, -1)
New_value

array([[  1.   ,   0.   ,   4.   ,  40.75 , -73.875,   3.   ,   1.   ,
          4.   ,   1.   ]])

In [106]:
test_predictions = rf_best.predict(New_value)

In [109]:
if test_predictions == 1:
      print("Drug and Alcohol Crime")
elif test_predictions == 2:
        print("Theft")
elif test_predictions == 3:
        print("Sex Crime")
elif test_predictions == 4:
          print("Assualt or Murder")
elif test_predictions == 5:
          print("Kidnapping and other crimes against children")
elif test_predictions == 6:
        print("City Law Violation")
    
 

Theft


In [110]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

In [113]:
def report_results(model, X, y):
    pred_proba = model.predict_proba(X)[:, 1]
    pred = model.predict(X)        

    auc = roc_auc_score(y, pred_proba)
    acc = accuracy_score(y, pred)
    f1 = f1_score(y, pred)
    prec = precision_score(y, pred)
    rec = recall_score(y, pred)
    cm = confusion_matrix(y, pred)
    result = {'auc': auc, 'f1': f1, 'acc': acc, 'precision': prec, 'recall': rec, 'confusion_matrix': cm}
    return result